In [ ]:
import pandas as pd
import os
from fastai.vision import *

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


from google.colab import drive
mount_directory='/content/gdrive'
picnic_data_dir=os.path.join(mount_directory,'My Drive/picnic-classifier/data')

# You will be require to enter an authorization code at this point
drive.mount(mount_directory)

In [ ]:
df = pd.read_csv(os.path.join(picnic_data_dir,'train.tsv'), sep='\t')

In [ ]:
resnet_size=224
databunch = ImageDataBunch.from_df(os.path.join(picnic_data_dir,'train'), df, size=resnet_size, ds_tfms=get_transforms(), valid_pct=0.05, num_workers=0)

In [ ]:
learner = cnn_learner(databunch, models.resnet101, metrics=accuracy)
learner.unfreeze()
learner.fit_one_cycle(8, slice(1e-5,3e-4))

In [ ]:
learner.save(os.path.join(picnic_data_dir,'picnic_model'))

In [ ]:
test_df = pd.read_csv(os.path.join(picnic_data_dir,'test.tsv'), sep='\t')
test_df['label'] = ''
test_dir = os.path.join(picnic_data_dir,'test')

for ix, row in test_df.iterrows():
  img_path = os.path.join(test_dir,row['file'])
  row['label'] = learner.predict(open_image(img_path))[0]
  
output_file=os.path.join(picnic_data_dir,'results.tsv')
test_df.to_csv(output_file, sep='\t', index=False)